In [1]:
import cv2
import pynput
from pynput.mouse import Controller, Button
from SIFT import find_target_image_in_screen as find
import random
import time
import os
import pyautogui

In [7]:
max_run_time = 3600*2
max_fail = 10
screen_width, screen_height = 2560, 1600

In [8]:
mouse = Controller()
keyboard = pynput.keyboard.Controller()
def click_like_human(x, y):
    x, y = int(x), int(y)
    current_x, current_y = mouse.position
    steps = random.randint(10, 30)  # Number of steps for gradual movement

    for step in range(steps):
        intermediate_x = current_x + (x - current_x) * (step + 1) / steps + random.uniform(-1, 1)
        intermediate_y = current_y + (y - current_y) * (step + 1) / steps + random.uniform(-1, 1)
        mouse.position = (intermediate_x, intermediate_y)
        time.sleep(random.uniform(0.01, 0.05))

    mouse.position = (x, y)
    time.sleep(0.3)
    mouse.click(Button.left)

def try_to_find(image_path, max_fail=max_fail, quit_on_fail=True, screenshot=None, region=None, tuning=False, ratio_threshold=0.5, response_threshold=0.02, good_match_threshold=0.7):
    fail = 0
    found = False
    while not found:
        found, bounding_box = find(image_path, screenshot=screenshot, region=region, tuning=tuning, ratio_threshold=ratio_threshold, response_threshold=response_threshold, good_match_threshold=good_match_threshold)
        fail += 1
        if fail > max_fail:
            if quit_on_fail:
                raise EOFError(f"Could not find image {image_path}")
            else:
                return False, None
    return True, bounding_box

def find_and_click(target_image, click_region=(0, 1, 0, 1), max_fail=max_fail, quit_on_fail=True, screenshot=None, region=None, tuning=False, ratio_threshold=0.5, response_threshold=0.02, good_match_threshold=0.7):
    x_min_click, x_max_click, y_min_click, y_max_click = click_region

    found, bounding_box = try_to_find(target_image, max_fail=max_fail, quit_on_fail=quit_on_fail, screenshot=screenshot, region=region, tuning=tuning, ratio_threshold=ratio_threshold, response_threshold=response_threshold, good_match_threshold=good_match_threshold)
    if found:
        x_min, y_min, width, height = bounding_box
        x, y = x_min + (width * (random.uniform(x_min_click, x_max_click))), y_min + (height * (random.uniform(y_min_click, y_max_click)))
        click_like_human(x, y)
    return found, bounding_box

def lose_the_game():
    # Step 1: 点击屏幕正中心
    click_like_human(int(screen_width/2), int(screen_height/2))

    # Step 2: 快速按 5 下 W
    for _ in range(5):
        keyboard.press('w')
        time.sleep(0.1)
        keyboard.release('w')
        time.sleep(0.15)

    # Step 3: 等待 50 秒
    time.sleep(50)

    # Step 4: 按下 ctrl_l + shift + A
    from pynput.keyboard import Key
    with keyboard.pressed(Key.ctrl_l):
        with keyboard.pressed(Key.shift):
            keyboard.press('a')
            time.sleep(0.1)
            keyboard.release('a')

    # Step 5: 按下 del 键 3 秒
    keyboard.press(Key.delete)
    time.sleep(3)
    keyboard.release(Key.delete)

def reset_mouse():
    mouse.position = (50, 50)


import pytesseract
import sys
if sys.platform == "darwin":
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    useGPU = True
    Tess_Path = r'/opt/homebrew/bin/tesseract'
else:
    useGPU = False
    Tess_Path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

pytesseract.pytesseract.tesseract_cmd = Tess_Path

In [9]:
screenshot_now = None

class StateMaChine:
    def __init__(self, handling_table, state_table):
        self.state_table = state_table
        self.handling_table = handling_table
        self.state = "unknown"

    def detect_state(self, screenshot=None, region=None, tuning=False, ratio_threshold=0.5, response_threshold=0.02, good_match_threshold=0.7):
        states = {}
        for imgPath in os.listdir("tar_image_AOE4"):
            if imgPath in self.state_table.keys():
                if imgPath == 'match_abstract.png':
                    good_match_threshold = 0.95
                found, bounding_box, _, _, _, _, _, good_match_ratio = find(os.path.join('tar_image_AOE4', imgPath), screenshot=screenshot, region=region, tuning=tuning, ratio_threshold=ratio_threshold, response_threshold=response_threshold, good_match_threshold=good_match_threshold, return_full_detail=True)
                if found:
                    if self.state_table[imgPath] == 'in game':
                        global screenshot_now
                        x_min, y_min, width, height = bounding_box
                        screenshot_now = pyautogui.screenshot()
                        try:
                            screenshot_cropped = screenshot_now.crop((x_min+55, y_min, x_min+width-100, y_min+height/4))
                            extracted_text = pytesseract.image_to_string(screenshot_cropped)
                            if any(s in extracted_text for s in ['0/0', 'o/o', 'o/0', '0/o']):
                                good_match_ratio = 0
                        except ValueError:
                            good_match_ratio = 0
                else:
                    good_match_ratio = 0

                states.update({imgPath : good_match_ratio})
        print(states)
        # Selects the state with heighest probability
        imgPath = max(states, key=states.get)
        if states[imgPath] == 0:
            self.state = "unknown"
            return False
        else:
            self.state = self.state_table[imgPath]
            return True

    def act(self, dummy_action = False):
        if self.state in self.handling_table.keys():
            if dummy_action:
                print(f"dummy_action: {self.state}")
            else:
                self.handling_table[self.state]()
        else:
            pass


In [10]:
state_table = {
    'match_abstract.png' : 'game ends',
    'play_again.png' : 'match abstract',
    'right_down_UI.png' :'in game',
    'search_game.png' : 'front page'
}

handling_table = {
    'game ends' : lambda : find_and_click('tar_image_AOE4/match_abstract.png', click_region=(0.1, 0.9, 0.1, 0.9), ratio_threshold=0.4, good_match_threshold=0.9, max_fail=10, quit_on_fail=False),
    'match abstract' : lambda : find_and_click('tar_image_AOE4/play_again.png', click_region=(0.1, 0.9, 0.1, 0.9), ratio_threshold=0.4, good_match_threshold=0.9, max_fail=10, quit_on_fail=False),
    'in game' : lambda : lose_the_game(),
    'front page' : lambda : find_and_click('tar_image_AOE4/search_game.png', click_region=(0.1, 0.9, 0.1, 0.9), max_fail=10, quit_on_fail=False),
    'unknown' : lambda : reset_mouse(),
}

In [11]:
time.sleep(2)
t0 = time.time()
sm = StateMaChine(handling_table, state_table)
while True:
    sm.detect_state(ratio_threshold=0.5, good_match_threshold=0.85)
    print(f"current state: {sm.state}")
    if (time.time() - t0 > max_run_time) and (sm.state == "game ends" or sm.state == "match abstract"):
        break
    sm.act(dummy_action=False)
    time.sleep(2)

{'match_abstract.png': 0, 'play_again.png': 0.5641025641025641, 'right_down_UI.png': 0, 'search_game.png': 0.949238578680203}
current state: front page
{'match_abstract.png': 0, 'play_again.png': 0.6666666666666666, 'right_down_UI.png': 0, 'search_game.png': 0}
current state: match abstract
{'match_abstract.png': 0, 'play_again.png': 0.6153846153846154, 'right_down_UI.png': 0, 'search_game.png': 0}
current state: match abstract
{'match_abstract.png': 0.5363128491620112, 'play_again.png': 0.5384615384615384, 'right_down_UI.png': 0, 'search_game.png': 0.5329949238578681}
current state: match abstract
{'match_abstract.png': 0.5307262569832403, 'play_again.png': 0.5641025641025641, 'right_down_UI.png': 0, 'search_game.png': 0}
current state: match abstract
{'match_abstract.png': 0.5195530726256983, 'play_again.png': 0, 'right_down_UI.png': 0, 'search_game.png': 0}
current state: game ends
{'match_abstract.png': 0, 'play_again.png': 0, 'right_down_UI.png': 0, 'search_game.png': 0}
current s

KeyboardInterrupt: 